In [5]:

import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

df = pd.read_csv('nuclear_data/impacts-of-energy-sources.csv')

df = df.drop(['Agricultural land use', 'Urban land use', 'Death rates', 'Year'], axis=1)

df = df.rename(columns={'Greenhouse gas emissions': 'Greenhouse gas emissions (kg CO2 eq.)', 
                        'Freshwater eutrophication': 'Freshwater eutrophication (kg P eq.)', 
                        'Ionising radiation':'Ionising radiation (kBq 235U eq)', 
                        'Water use':'Water use (m3)',
                        'Metal and mineral requirements':'Metal and mineral requirements (kg Sb eq.)',
                        'Non-carcinogenic toxicity':'Non-carcinogenic toxicity (CTUh)',
                        'Carcinogenic toxicity':'Carcinogenic toxicity (CTUh)',
                        'Total land use':'Total land use (points)',
                        'Uranium':'Uranium (kg Sb eq.)'})

df['Entity'] = df['Entity'].replace('Coal with carbon capture (CCS)', 'Coal')

df['Entity'] = df['Entity'].replace('Concentrating solar (tower)', 'Concentrating Solar')
df['Entity'] = df['Entity'].replace('Concentrating solar (trough)', 'Concentrating Solar')

df['Entity'] = df['Entity'].replace('Solar PV, cadmium (roof)', 'Solar PV')
df['Entity'] = df['Entity'].replace('Solar PV, cadmium (on-ground)', 'Solar PV')
df['Entity'] = df['Entity'].replace('Solar PV, silicon (on-ground)', 'Solar PV')
df['Entity'] = df['Entity'].replace('Solar PV, silicon (roof)', 'Solar PV')

df['Entity'] = df['Entity'].replace('Gas with carbon capture (CCS)', 'Gas')

df['Entity'] = df['Entity'].replace('Offshore wind', 'Wind')
df['Entity'] = df['Entity'].replace('Onshore wind', 'Wind')

df_combined = df.drop(columns=['Aluminium', 'Chromium', 'Cobalt', 'Copper', 'Manganese', 'Molybdenum', 'Nickel', 'Silicon', 'Zinc'])
df_combined['Human toxicity (CTUh)'] = df_combined['Non-carcinogenic toxicity (CTUh)'] + df_combined['Carcinogenic toxicity (CTUh)']
df_combined = df_combined.drop(columns=['Non-carcinogenic toxicity (CTUh)', 'Carcinogenic toxicity (CTUh)'])
df_combined = df_combined[['Entity', 'Greenhouse gas emissions (kg CO2 eq.)', 
                           'Freshwater eutrophication (kg P eq.)', 
                           'Ionising radiation (kBq 235U eq)', 
                           'Water use (m3)', 
                           'Total land use (points)', 
                           'Human toxicity (CTUh)', 
                           'Metal and mineral requirements (kg Sb eq.)', 
                           'Uranium (kg Sb eq.)']]

df_combined = df_combined.groupby('Entity', as_index=False).sum()

color_map = {
    'Coal': '#636EFA',
    'Concentrating Solar': '#EF553B',
    'Gas': '#00CC96',
    'Hydropower': '#AB63FA',
    'Nuclear': '#FFA15A',
    'Solar PV': '#19D3F3',
    'Wind': '#FF6692'
}

metrics6 = df_combined.columns[1:]
fig11 = make_subplots(
    rows=3, 
    cols=3,
    subplot_titles=metrics6,
    horizontal_spacing=0.15,
    vertical_spacing=0.2
)

for i, metric in enumerate(metrics6, 1):
    row = (i-1) // 3 + 1
    col = (i-1) % 3 + 1
    
    for entity in df_combined['Entity']:
        fig11.add_trace(
            go.Bar(
                x=[entity],
                y=df_combined[df_combined['Entity']==entity][metric],
                name=entity,
                marker_color=color_map[entity],
                showlegend=(i == 1)
            ),
            row=row,
            col=col
        )

fig11.update_layout(
    title_text='Energy Source Environmental Impact Comparison',
    height=1300,
    width=1200,
    legend_title_text='Energy Source'
)

fig11.update_yaxes(type="log", row=3, col=3) 

fig11.show()